In [42]:
import os
import sys
import pandas as pd
sys.path.append('/Users/dane/src/datatools')

In [26]:
%cd /Users/dane/src/datatools/ipy/notebooks/regtest/dev
df = pd.read_pickle('../data/v14_b8600_b8658.pickle')

/Users/dane/src/datatools/ipy/notebooks/regtest/dev


In [27]:
df.head(1)

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut,run_name
0,1,B8622,REV10,0,TESTEXAMPLES,1.284,1.45576,0,153,B8622.REV10


In [29]:
test_times = pd.DataFrame(columns=['sim_mean', 'sim_rsd', 'el_mean', 'el_rsd'], index=['project'])

In [ ]:
results = []
for project, runs in df.groupby('project'):
    sim_mean = runs.sim_time.mean()
    std = runs.sim_time.std()
    sim_rsd = std / sim_mean
    outliers = len(runs[runs.sim_time > (sim_mean + 2 * std)])
    el_mean = runs.elapsed_time.mean()
    el_rsd = runs.elapsed_time.std() / el_mean
    results.append( (project, sim_mean, sim_rsd, outliers, el_mean, el_rsd) )

test_times = pd.DataFrame.from_records(results,
                                       columns=['project', 'sim_mean', 'sim_rsd',
                                                'outliers', 'el_mean', 'el_rsd'])


In [36]:
# RSD is not that useful on very small times
mask_time = test_times.sim_mean > 0.25
mask_rsd = test_times.sim_rsd > 0.5
test_times[mask_time & mask_rsd].head()

,sim_mean,sim_rsd,el_mean,el_rsd
57,0.2830748,0.55196,0.9792544,0.09146986
658,0.3840597,1.846811,68.65402,0.06386194
1662,0.3951804,0.7425962,3.620965,0.09387406
1694,0.3624825,1.506066,1.1799,0.4690344
3978,0.258379,0.5288159,1.025048,0.1298674
4585,0.5816937,0.5344698,1.100097,0.7808752
4586,0.3204727,2.455602,0.5992777,1.648316
4760,1.20065,1.690065,1.305103,1.554686
4899,0.3509105,0.5512655,0.5095165,0.3871582
4929,0.5119119,2.232192,0.6819583,1.6754


In [39]:
df[(df.project==4760) & (df.sim_time > 2)]

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut,run_name
4694233,1,B8658,REV4,4760,TEST3M,55.383,55.4829,1,411,B8658.REV4


In [49]:
from regtest.ingest_log_files import StrMap
pmap = StrMap.read('../../../../regtest/project_name')

In [67]:
# build a reverse map and shorten names to useful part
rmap = {v:k.replace('\\', '/')[37:] for k, v in pmap.items()}

In [78]:
# add names to the times
test_times['name'] = test_times.index.map(rmap.get)

In [79]:
# sort by desc rsd for significant sim times
test_times[test_times.sim_mean > 0.5].sort(['sim_rsd'], ascending=[0])

,sim_mean,sim_rsd,el_mean,el_rsd,name,sim_stdmed
7158,1.750333,7.631726,2.082314,6.413051,buildtests/InheritedParams/UCT_parameter_passi...,11.555416
10152,15.01908,2.788927,27.70709,2.839702,scripting/API/pcb_import/odb/test_projects/neg...,406.671085
6451,0.7061413,2.42646,0.9311852,1.840344,buildtests/EM_Tests/SPP_per_layer/tt17188_TTdi...,2.689833
4929,0.5119119,2.232192,0.6819583,1.6754,buildtests/ALFREDO/MOST/NMOS_VS_PMOS.emp,2.452115
6679,0.6363538,2.071892,0.8200595,1.616672,buildtests/EXPORT_TRANSFORMS_APLAC/MEXTRAM504/...,2.257631
4760,1.20065,1.690065,1.305103,1.554686,buildtests/ALFREDO/HICUM_L0/TwoStages.emp,1.802111
6758,1.351933,1.651437,1.560647,1.430337,buildtests/Extraction/extraction_groups/Group_...,1.759363
6520,1.371435,1.057461,1.525906,0.9502597,buildtests/Examples/60 Examples/Harmonic Balan...,1.109594
10138,0.6779969,0.9448658,0.7466373,0.858222,labview/TT16791.emp,2.399311
8514,2.955653,0.8206742,3.158162,0.7810512,buildtests/SysBlocks/PHARRAY.emp,2.976231


In [68]:
a = rmap.get
a(1000)

'performance/axiem/mesh/TQNC_TQPED_Coupler/TQNC_TQPED_Coupler_Med_Decimate.emp'